# Exercise 02: Multi-class Classification 
In this exercise, you will train a deep model on the CIFAR10 from the scratch using PyTorch. The following tasks should be done:
- Task 1: per batch training/testing
- Task 2: Instance inference and visualization

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from PIL import Image

### Hyperparameters

In [2]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 40
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5


### Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset


In [4]:
# cifar10 transform
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar10_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Files already downloaded and verified
Files already downloaded and verified


### Model

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

### Optimizer

In [7]:
# optimizer = optim.Adam(model.parameters())

### Task 1: per batch training/testing
---

Please denfine two function named ``train_batch`` and ``test_batch``. These functions are essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Define the loss function i.e [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).
2. Take the image as the input and generate the output using the pre-defined SimpleNet.
3. Calculate the loss between the output and the corresponding label using the loss function.

In [8]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, output, target):
        # 计算交叉熵损失
        ce_loss = F.cross_entropy(output, target, reduction='none')

        # 计算概率
        pt = torch.exp(-ce_loss)

        # 计算Focal Loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        # 加权Focal Loss
        if self.alpha is not None:
            assert len(self.alpha) == output.size(1), "Alpha must have the same length as the number of classes."
            alpha = self.alpha[target]
            focal_loss = focal_loss * alpha

        return focal_loss.mean()

In [9]:
##################### Write your answer here ##################
# Define the loss function

#     MAE (L1) Loss:
criterion1 = nn.L1Loss()

#     CE (Cross-Entropy) Loss
criterion2 = nn.CrossEntropyLoss()
    
#     Focal Loss (gamma=0.5)
criterion3 = FocalLoss(gamma=0.5, alpha=None)

#     Focal Loss (gamma=2)
criterion4 = FocalLoss(gamma=2, alpha=None)

###############################################################

In [10]:
def train_batch(model, image, target, criterion):
    """
    Perform one training batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to train.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated by the defined loss function loss_fn().
    """
    output = model(image)
#     print(output)
#     print(target)
    ##################### Write your answer here ##################
#     MAE (L1) Loss:
#     loss = criterion1(output,F.one_hot(target,num_classes = 10).float())

#     CE (Cross-Entropy) Loss
#     loss = criterion2(output,target)
    
#     Focal Loss (gamma=0.5)
#     loss = criterion3(output,target)
    
#     Focal Loss (gamma=2)
#     loss = criterion4(output,target)
    loss = criterion(output,target)

    ###############################################################

    return output, loss

In [11]:

def test_batch(model, image, target,criterion):
    """
    Perform one testing batch iteration.

    Args:
        model (torch.nn.Module): The machine learning model to evaluate.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.

    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated for the batch.
    """
    output = model(image)
    
    ##################### Write your answer here ##################
#     MAE (L1) Loss:
#     loss = criterion1(output,F.one_hot(target,num_classes = 10).float())

#     CE (Cross-Entropy) Loss
#     loss = criterion2(output,target)
                      
#     Focal Loss (gamma=0.5)
#     loss = criterion3(output,target)
    
#     Focal Loss (gamma=2)
#     loss = criterion4(output,target)

    loss = criterion(output,target)

    ###############################################################

    return output, loss

### Model Training

In [12]:
def model_training(NUM_EPOCHS,EVAL_INTERVAL, SAVE_DIR,model,criterion, loss_name):
    
    model.to(device)
#     Adam
    optimizer = optim.Adam(model.parameters())

    training_loss_Adam = []
    training_acc_Adam = []
    testing_loss_Adam = []
    testing_acc_Adam = []
    testing_loss_Adam_S = []
    testing_acc_Adam_S = []

    for epoch in range(NUM_EPOCHS):
        model.train()
        torch.cuda.empty_cache()

        ##########################
        ### Training
        ##########################

        running_cls_loss = 0.0
        running_cls_corrects = 0

        for batch_idx, (image, target) in enumerate(train_dataloader):

            image = image.to(device)
            target = target.to(device)

            # train model
            
            if loss_name == "MAE (L1) Loss.csv":
                outputs, loss = train_batch(model, image, F.one_hot(target,num_classes = 10).float(),criterion)
            else:
                outputs, loss = train_batch(model, image, target,criterion)
                
            _, preds = torch.max(outputs, 1)


            loss_data = loss.data.item()
            if np.isnan(loss_data):
                raise ValueError('loss is nan while training')
            running_cls_loss += loss.item()
            running_cls_corrects += torch.sum(preds == target.data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss = running_cls_loss / len(train_set)
        epoch_acc = running_cls_corrects.double() / len(train_set)

        print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        training_loss_Adam.append(epoch_loss)
        training_acc_Adam.append(epoch_acc.cpu().detach().numpy())

        # change learning rate
    #     scheduler.step()


        ##########################
        ### Testing
        ##########################
        # # eval model during training or in the last epoch
        if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch +1) == NUM_EPOCHS:
            print('Begin test......')
            model.eval()

            val_loss = 0.0
            val_corrects = 0

            # origin
            for batch_idx, (image, target) in enumerate(test_dataloader):

                image = image.to(device)
                target = target.to(device)
                
                if loss_name == "MAE (L1) Loss.csv":
                    outputs, loss = test_batch(model, image, F.one_hot(target,num_classes = 10).float(),criterion)
                else:
                    outputs, loss = test_batch(model, image, target,criterion)              

                # test model
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / len(test_set)
            val_acc = val_corrects.double() / len(test_set)
            print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
            testing_loss_Adam.append(val_loss)
            testing_acc_Adam.append(val_acc.cpu().detach().numpy())

            # noise
            for batch_idx, (image, target) in enumerate(test_dataloader):
                image_noise = image+ torch.randn(image.size())
                image_noise = image_noise.to(device)
                target = target.to(device)
                
                # test model                
                if loss_name == "MAE (L1) Loss.csv":
                    outputs, loss = test_batch(model, image_noise, F.one_hot(target,num_classes = 10).float(),criterion)
                else:
                    outputs, loss = test_batch(model, image_noise, target,criterion)

                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / len(test_set)
            val_acc = val_corrects.double() / len(test_set)
            testing_loss_Adam_S.append(val_loss)
            testing_acc_Adam_S.append(val_acc.cpu().detach().numpy())


            # save the model in last epoch
            if (epoch +1) == NUM_EPOCHS:

                state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch+1),
                }

                # check the dir
                if not os.path.exists(SAVE_DIR):
                    os.makedirs(SAVE_DIR)

                # save the state
                torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch+1))))

                
#     SGD

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)
    training_loss_SGD = []
    training_acc_SGD = []
    testing_loss_SGD = []
    testing_acc_SGD = []
    testing_loss_SGD_S = []
    testing_acc_SGD_S = []

    for epoch in range(NUM_EPOCHS):
        model.train()
        torch.cuda.empty_cache()

        ##########################
        ### Training
        ##########################

        running_cls_loss = 0.0
        running_cls_corrects = 0

        for batch_idx, (image, target) in enumerate(train_dataloader):

            image = image.to(device)
            target = target.to(device)
            
            
            # train model
            if loss_name == "MAE (L1) Loss.csv":
                outputs, loss = train_batch(model, image, F.one_hot(target,num_classes = 10).float(),criterion)
            else:
                outputs, loss = train_batch(model, image, target,criterion)           

            _, preds = torch.max(outputs, 1)


            loss_data = loss.data.item()
            if np.isnan(loss_data):
                raise ValueError('loss is nan while training')
            running_cls_loss += loss.item()
            running_cls_corrects += torch.sum(preds == target.data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss = running_cls_loss / len(train_set)
        epoch_acc = running_cls_corrects.double() / len(train_set)

        print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        training_loss_SGD.append(epoch_loss)
        training_acc_SGD.append(epoch_acc.cpu().detach().numpy())

        # change learning rate
        scheduler.step()


        ##########################
        ### Testing
        ##########################
        # # eval model during training or in the last epoch
        if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch +1) == NUM_EPOCHS:
            print('Begin test......')
            model.eval()

            ## origin
            val_loss = 0.0
            val_corrects = 0

            for batch_idx, (image, target) in enumerate(test_dataloader):

                image = image.to(device)
                target = target.to(device)
                
                if loss_name == "MAE (L1) Loss.csv":
                    outputs, loss = test_batch(model, image, F.one_hot(target,num_classes = 10).float(),criterion)
                else:
                    outputs, loss = test_batch(model, image, target,criterion)  
                    
                # test model
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / len(test_set)
            val_acc = val_corrects.double() / len(test_set)
            print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
            testing_loss_SGD.append(val_loss)
            testing_acc_SGD.append(val_acc.cpu().detach().numpy())

            ## noise
            val_loss = 0.0
            val_corrects = 0

            for batch_idx, (image, target) in enumerate(test_dataloader):
                image_noise = image+ torch.randn(image.size())
                image_noise = image_noise.to(device)
                target = target.to(device)

                if loss_name == "MAE (L1) Loss.csv":
                    outputs, loss = test_batch(model, image_noise, F.one_hot(target,num_classes = 10).float(),criterion)
                else:
                    outputs, loss = test_batch(model, image_noise, target,criterion)  
                    
                # test model
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / len(test_set)
            val_acc = val_corrects.double() / len(test_set)
            testing_loss_SGD_S.append(val_loss)
            testing_acc_SGD_S.append(val_acc.cpu().detach().numpy())



            # save the model in last epoch
            if (epoch +1) == NUM_EPOCHS:

                state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch+1),
                }

                # check the dir
                if not os.path.exists(SAVE_DIR):
                    os.makedirs(SAVE_DIR)

                # save the state
                torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch+1))))
    
    df = pd.DataFrame({
        "training_loss_SGD":training_loss_SGD,
        "training_acc_SGD":training_acc_SGD,
        "testing_loss_SGD":testing_loss_SGD,
        "testing_acc_SGD":testing_acc_SGD,
        "testing_loss_SGD_S":testing_loss_SGD_S,
        "testing_acc_SGD_S":testing_acc_SGD_S,
        "training_loss_Adam":training_loss_Adam,
        "training_acc_Adam":training_acc_Adam,
        "testing_loss_Adam":testing_loss_Adam,
        "testing_acc_Adam":testing_acc_Adam,
        "testing_loss_Adam_S":testing_loss_Adam_S,
        "testing_acc_Adam_S":testing_acc_Adam_S,
    })
    df.to_csv("data_new//"+loss_name, index=False)  

In [13]:
#     MAE (L1) Loss:
#     loss = criterion1(output,F.one_hot(target,num_classes = 10).float())
model_training(NUM_EPOCHS,EVAL_INTERVAL, SAVE_DIR,ConvNet(),criterion1, "MAE (L1) Loss.csv")

Epoch: 1/40 Train Loss: 0.0008 Acc: 0.1082
Begin test......
Test Loss: 0.0008 Acc: 0.1147
Epoch: 2/40 Train Loss: 0.0008 Acc: 0.1080
Begin test......
Test Loss: 0.0008 Acc: 0.1009
Epoch: 3/40 Train Loss: 0.0008 Acc: 0.1049
Begin test......
Test Loss: 0.0008 Acc: 0.1017
Epoch: 4/40 Train Loss: 0.0008 Acc: 0.1000
Begin test......
Test Loss: 0.0008 Acc: 0.1016
Epoch: 5/40 Train Loss: 0.0008 Acc: 0.1009
Begin test......
Test Loss: 0.0008 Acc: 0.1029
Epoch: 6/40 Train Loss: 0.0008 Acc: 0.1022
Begin test......
Test Loss: 0.0008 Acc: 0.1225
Epoch: 7/40 Train Loss: 0.0008 Acc: 0.1000
Begin test......
Test Loss: 0.0008 Acc: 0.1033
Epoch: 8/40 Train Loss: 0.0008 Acc: 0.1016
Begin test......
Test Loss: 0.0008 Acc: 0.0964
Epoch: 9/40 Train Loss: 0.0008 Acc: 0.1012
Begin test......
Test Loss: 0.0008 Acc: 0.1006
Epoch: 10/40 Train Loss: 0.0008 Acc: 0.1018
Begin test......
Test Loss: 0.0008 Acc: 0.1227
Epoch: 11/40 Train Loss: 0.0008 Acc: 0.1034
Begin test......
Test Loss: 0.0008 Acc: 0.0992
Epoch: 1

In [14]:
#     CE (Cross-Entropy) Loss
#     loss = criterion2(output,target)
model_training(NUM_EPOCHS,EVAL_INTERVAL, SAVE_DIR,ConvNet(),criterion2, "CE (Cross-Entropy) Loss.csv")   

Epoch: 1/40 Train Loss: 0.0152 Acc: 0.2872
Begin test......
Test Loss: 0.0132 Acc: 0.3964
Epoch: 2/40 Train Loss: 0.0131 Acc: 0.3927
Begin test......
Test Loss: 0.0123 Acc: 0.4456
Epoch: 3/40 Train Loss: 0.0125 Acc: 0.4251
Begin test......
Test Loss: 0.0116 Acc: 0.4775
Epoch: 4/40 Train Loss: 0.0120 Acc: 0.4482
Begin test......
Test Loss: 0.0113 Acc: 0.4870
Epoch: 5/40 Train Loss: 0.0117 Acc: 0.4624
Begin test......
Test Loss: 0.0110 Acc: 0.5054
Epoch: 6/40 Train Loss: 0.0115 Acc: 0.4728
Begin test......
Test Loss: 0.0107 Acc: 0.5194
Epoch: 7/40 Train Loss: 0.0113 Acc: 0.4818
Begin test......
Test Loss: 0.0105 Acc: 0.5321
Epoch: 8/40 Train Loss: 0.0112 Acc: 0.4906
Begin test......
Test Loss: 0.0104 Acc: 0.5305
Epoch: 9/40 Train Loss: 0.0111 Acc: 0.4949
Begin test......
Test Loss: 0.0102 Acc: 0.5460
Epoch: 10/40 Train Loss: 0.0109 Acc: 0.5030
Begin test......
Test Loss: 0.0101 Acc: 0.5467
Epoch: 11/40 Train Loss: 0.0109 Acc: 0.5062
Begin test......
Test Loss: 0.0100 Acc: 0.5517
Epoch: 1

In [15]:
#     Focal Loss (gamma=0.5)
#     loss = criterion3(output,target)
model_training(NUM_EPOCHS,EVAL_INTERVAL, SAVE_DIR,ConvNet(),criterion3, "Focal Loss (gamma=0.5).csv")                 

Epoch: 1/40 Train Loss: 0.0141 Acc: 0.2929
Begin test......
Test Loss: 0.0123 Acc: 0.3836
Epoch: 2/40 Train Loss: 0.0123 Acc: 0.3718
Begin test......
Test Loss: 0.0112 Acc: 0.4334
Epoch: 3/40 Train Loss: 0.0116 Acc: 0.4048
Begin test......
Test Loss: 0.0107 Acc: 0.4580
Epoch: 4/40 Train Loss: 0.0111 Acc: 0.4206
Begin test......
Test Loss: 0.0103 Acc: 0.4664
Epoch: 5/40 Train Loss: 0.0108 Acc: 0.4377
Begin test......
Test Loss: 0.0099 Acc: 0.4885
Epoch: 6/40 Train Loss: 0.0105 Acc: 0.4543
Begin test......
Test Loss: 0.0099 Acc: 0.4901
Epoch: 7/40 Train Loss: 0.0102 Acc: 0.4691
Begin test......
Test Loss: 0.0094 Acc: 0.5122
Epoch: 8/40 Train Loss: 0.0101 Acc: 0.4754
Begin test......
Test Loss: 0.0093 Acc: 0.5256
Epoch: 9/40 Train Loss: 0.0099 Acc: 0.4841
Begin test......
Test Loss: 0.0092 Acc: 0.5351
Epoch: 10/40 Train Loss: 0.0098 Acc: 0.4891
Begin test......
Test Loss: 0.0091 Acc: 0.5347
Epoch: 11/40 Train Loss: 0.0097 Acc: 0.4984
Begin test......
Test Loss: 0.0089 Acc: 0.5421
Epoch: 1

In [16]:
#     Focal Loss (gamma=2)
#     loss = criterion4(output,target)
model_training(NUM_EPOCHS,EVAL_INTERVAL, SAVE_DIR,ConvNet(),criterion4, "Focal Loss (gamma=2).csv")                 

Epoch: 1/40 Train Loss: 0.0110 Acc: 0.3083
Begin test......
Test Loss: 0.0091 Acc: 0.4136
Epoch: 2/40 Train Loss: 0.0092 Acc: 0.3908
Begin test......
Test Loss: 0.0083 Acc: 0.4489
Epoch: 3/40 Train Loss: 0.0086 Acc: 0.4174
Begin test......
Test Loss: 0.0078 Acc: 0.4726
Epoch: 4/40 Train Loss: 0.0082 Acc: 0.4376
Begin test......
Test Loss: 0.0075 Acc: 0.4812
Epoch: 5/40 Train Loss: 0.0080 Acc: 0.4457
Begin test......
Test Loss: 0.0072 Acc: 0.4975
Epoch: 6/40 Train Loss: 0.0078 Acc: 0.4566
Begin test......
Test Loss: 0.0071 Acc: 0.4985
Epoch: 7/40 Train Loss: 0.0077 Acc: 0.4643
Begin test......
Test Loss: 0.0069 Acc: 0.5140
Epoch: 8/40 Train Loss: 0.0075 Acc: 0.4717
Begin test......
Test Loss: 0.0068 Acc: 0.5167
Epoch: 9/40 Train Loss: 0.0075 Acc: 0.4768
Begin test......
Test Loss: 0.0067 Acc: 0.5290
Epoch: 10/40 Train Loss: 0.0074 Acc: 0.4837
Begin test......
Test Loss: 0.0066 Acc: 0.5299
Epoch: 11/40 Train Loss: 0.0073 Acc: 0.4877
Begin test......
Test Loss: 0.0066 Acc: 0.5264
Epoch: 1

In [17]:
# plt.plot(testing_acc_Adam)
# plt.plot(testing_acc_SGD)

### Task 2: Instance inference
---
The task is to visualizes an image along with model prediction and class probabilities.

**To do**: 
1. Calculate the prediction and the probabilities for each class.
         

In [18]:
# inputs, classes = next(iter(test_dataloader))
# input = inputs[0]

In [19]:
# ##################### Write your answer here ##################
# # input: image, model
# # outputs: predict_label, probabilities
# # predict_label is the index (or label) of the class with the highest probability from the probabilities.
# ###############################################################
# input = input.to(device)
# probabilities = model(input)[0]
# predict_label = torch.argmax(probabilities)

# print(probabilities)
# print(predict_label)

In [20]:
# predicted_class = class_names[predict_label.item()]
# predicted_probability = probabilities[predict_label].item()
# input = input.cpu()
# image = input.numpy().transpose((1, 2, 0))
# plt.imshow(image)
# plt.text(17, 30, f'Predicted Class: {predicted_class}\nProbability: {predicted_probability:.2f}', 
#             color='white', backgroundcolor='black', fontsize=8)
# plt.show()

# # Print probabilities for each class
# print('Print probabilities for each class:')
# for i in range(len(class_names)):
#     print(f'{class_names[i]}: {probabilities[i].item():.4f}')